<a href="https://colab.research.google.com/github/SalarNouri/Advanced-Deep-Learning/blob/main/models_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install the What-If Tool widget if running in colab {display-mode: "form"}

# If running in colab then pip install, otherwise no need.

try:
  import google.colab
  !pip install --upgrade tensorflow>=2.0.0 witwidget 
except Exception:
  pass

In [ ]:
#@title Import nesecassary libraries

import json
import pandas as pd
import numpy as np
import tensorflow as tf

from google.protobuf import json_format
from tensorflow.keras import models
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder


In [ ]:
#@title Define helper functions for dataset conversion

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None) -> tf.train.Example:
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    examples.append(example)
  return examples

# Convert list of tf.Examples into dataframe

def tfexample_to_df(example:tf.train.Example) -> pd.DataFrame:
    df_example = pd.DataFrame([])
    idx = 0
    for item in features:
      example_json = json_format.MessageToJson(item)
      data = json.loads(example_json)
      z = data['features']['feature']
      for key, value in z.items():
        new_value = ( list(value.values())[0] ).values()
        z[key] = float(np.array(list(new_value)[0]))
      x = pd.DataFrame(z, index=[0])
      df_example = df_example.append(x, ignore_index=True)
    return df_example

In [ ]:
#@title Features Name

features_name = {'price': 1,
 'hour': 0,
 'weekday': 0,
 'estimated_distance': 0,
 'with_waiting': 0,
 'price_to_est_dist': 0,
 'has_extra_destination': 0,
 'origin_district_id': 0,
 'destination_district_id': 0,
 'extra_destination_district_id': 0,
 'origin_in_even_odd': 0,
 'destination_in_even_odd': 0,
 'extra_destination_in_even_odd': 0,
 'origin_in_traffic_zone': 0,
 'destination_in_traffic_zone': 0,
 'destination_city_id': 1,
 'vehicle_model_id': 0,
 'vehicle_color_id': 0,
 'driver_service_type': 0,
 'rating': 0,
 'oldest': 0,
 'driver_city_id': 0,
 'has_traffic_license': 0,
 'driver_score': 1,
 'score_to_est_dist': 0,
 'week_offers': 1,
 'previous_offers': 0,
 'offers_std': 1,
 'week_accepts': 1,
 'previous_accepts': 1,
 'accepts_std': 1,
 'week_accepts_valid': 1,
 'accepts_valid_std': 1,
 'week_rides': 1,
 'rides_std': 1,
 'week_accept_rates': 1,
 'previous_a_rate': 1,
 'accept_rates_std': 1,
 'week_finish_rates': 1,
 'previous_f_rates': 0,
 'finish_rates_std': 1,
 'week_t_cancel_rides': 1,
 'week_t_cancel_rates': 0,
 't_cancel_rates_std': 1,
 'week_cancel_rates_by_d': 0,
 'cancel_rates_by_d_std': 0,
 'week_cancel_rates_by_p': 0,
 'week_cancel_rides_by_d': 1,
 'week_cancel_rides_by_p': 1,
 'week_t_ma': 0,
 'previous_t_ma': 0,
 'week_per_distance': 1,
 'per_distance_std': 1,
 'est_distance_to_avg_rides': 0,
 'week_income': 0,
 'week_income_ride': 1,
 'income_ride_std': 1,
 'price_to_week_income_ride': 0,
 't_week_active': 0,
 't_week_available': 0,
 'driver_segment': 1,
 'finished':1}

In [ ]:
#@title Read and Modify dataset

data =  pd.read_json(open("sample_data_isfahan_for_lstm.json", "r", encoding="utf8"))
data = data.astype(float)
labels = np.array(data.pop('finished'))
data = data[data.columns.intersection(list(features_name.keys()))]
# features = np.array(data)
features = df_to_examples(data)

In [ ]:
#@title Untar/Unzip models

# !tar -xvf  '/content/isfahan_model.tar.gz'

!tar -xvf  '/content/isfahan_models1.tar.gz'

In [ ]:
#@title  Metrics: F1 and MCC

def f1(y_true, y_pred):
    def recall_m(label, predicted):
        tp = keras_backend.sum(keras_backend.round(keras_backend.clip(label * predicted, 0, 1)))
        positives = keras_backend.sum(keras_backend.round(keras_backend.clip(label, 0, 1)))

        return tp / (positives + keras_backend.epsilon())

    def precision_m(label, predicted):
        true_positive = keras_backend.sum(keras_backend.round(keras_backend.clip(label * predicted, 0, 1)))
        predicted_positive = keras_backend.sum(keras_backend.round(keras_backend.clip(predicted, 0, 1)))
        return true_positive / (predicted_positive + keras_backend.epsilon())

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2 * ((precision * recall) / (precision + recall + keras_backend.epsilon()))


def mcc(y_true, y_pred):
    y_pred_pos = keras_backend.round(keras_backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = keras_backend.round(keras_backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = keras_backend.sum(y_pos * y_pred_pos)
    tn = keras_backend.sum(y_neg * y_pred_neg)

    fp = keras_backend.sum(y_neg * y_pred_pos)
    fn = keras_backend.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = keras_backend.sqrt((tp + fp) * (tn + fn) * (tn + fp) * (tp + fn))
    return numerator / (denominator + keras_backend.epsilon())

In [ ]:
#@title Load the keras models

model_history1 = models.load_model('/content/build/', custom_objects={'f1':f1, 'mcc':mcc})
model_history2 = models.load_model('/content/build1/', custom_objects={'f1':f1, 'mcc':mcc}) 

In [ ]:
#@title Define custom prediction functions so that WIT infers using keras models

# WIT predict functions:

def custom_predict_1(examples_to_infer):
  predictions = model_history1.predict(tfexample_to_df(examples_to_infer))
  return predictions


def custom_predict_2(examples_to_infer):
  predictions = model_history2.predict(tfexample_to_df(examples_to_infer))
  return predictions

In [ ]:
#@title Invoke What-If Tool for the data and two models

Num_DataPoints: int = 1000  #@param {type: "number"}
tool_height_in_px: int = 720  #@param {type: "number"}

# Setup the tool with the test examples and the trained classifier

config_builder = WitConfigBuilder(features[:Num_DataPoints]).set_custom_predict_fn(
  custom_predict_1).set_compare_custom_predict_fn(custom_predict_2)

wv = WitWidget(config_builder, height=tool_height_in_px)